# Geocoding on DAS

### Prep

In [ ]:
import arcgis
from arcgis.gis import GIS
from arcgis.geocoding import geocode

#For batch geocoding:
from arcgis.geocoding import batch_geocode
import pandas as pd

### Authenticate 

In [ ]:
#Replace client_id: https://statcan.github.io/gae-eag/english/portal/#arcgis-enterprise-via-api

gis = GIS("https://geoanalytics.cloud.statcan.ca/portal", client_id='#########', verify=True)
print("Logged in as: " + gis.properties.user.username)

### Single Address

In [ ]:
#Sinlge Address:
address = "150 Tunney's Pasture Driveway, Ottawa, ON K1A 0T6"
results =geocode(address)
print(results[0]['location'])

### Multiple Addresses

In [ ]:
#Multiple Address:
addresses = ["150 Tunney's Pasture Driveway, Ottawa, Ontario, K1A 0T6", 
             "1770 Market Street, HALIFAX, Nova Scotia, B3J 3M3", 
             "200 René Lévesque Blvd. W., Guy Favreau Complex, 4th Floor, East Tower, MONTRÉAL, Quebec, H2Z 1X4", 
             "25 St. Clair Avenue East, TORONTO, Ontario, M4T 1M4",
             "Park Square, 9th Floor, 10001 Bellamy Hill, EDMONTON, Alberta, T5J 3B6",
             "Library Square Office Tower, 600-300 West Georgia Street, VANCOUVER, British Columbia, V6B 6C7"
            ]

for address in addresses:
    results = geocode(address)
    print(results[0]['location'])

### Batch from CSV 

#### Geocode by creating a new CSV, leaving source data untouched:

In [ ]:
# Read in the CSV file as a pandas DataFrame, REPLACE <PATH TO SOURCE> (csv must not be locked/in use)
df = pd.read_csv("<PATH TO SOURCE CSV>")

# Create a list of dictionaries with address fields
addresses = df.to_dict("records")

# Batch geocode the list of addresses
results = batch_geocode(addresses)

# Create a list of dictionaries with x and y coordinates
locations = [{"x": result["location"]["x"], "y": result["location"]["y"]} for result in results]

# Add the x and y coordinates to the original DataFrame
df["x"] = [location["x"] for location in locations]
df["y"] = [location["y"] for location in locations]

# Save the updated DataFrame as a new CSV file, REPLACE <PATH TO NEW CSV>
df.to_csv("<PATH TO NEW CSV>", index=False)

#### Geocode by appending XY columns to source data:

In [ ]:
#Create variable for source data, REPLACE <PATH TO SOURCE>
source= "<PATH TO SOURCE>"

# Read in the CSV file as a pandas DataFrame (csv must not be locked/in use)
df = pd.read_csv(source)

# Create a list of dictionaries with address fields
addresses = df.to_dict("records")

# Batch geocode the list of addresses
results = batch_geocode(addresses)

# Create a list of dictionaries with x and y coordinates
locations = [{"x": result["location"]["x"], "y": result["location"]["y"]} for result in results]

# Add the x and y coordinates to the original DataFrame
for i, location in enumerate(locations):
    df.at[i, "x"] = location["x"]
    df.at[i, "y"] = location["y"]

# Save the updated DataFrame to the original CSV file
df.to_csv(source, index=False)